# TinyLlama-1.1B-Chat Test
Bu notebook Hugging Face'ten TinyLlama-1.1B-Chat-v1.0 modelini yükler ve test eder.

**✅ Avantajlar:**
- Küçük model (1.1B parametre)
- CPU'da çalışır (GPU şart değil)
- Hızlı yükleme ve cevap

In [ ]:
# Gerekli kütüphaneleri yükle (TinyLlama için quantization gerekmez)
%pip install -q transformers torch accelerate

In [ ]:
# Import'lar
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Model ayarları
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# GPU kontrolü (opsiyonel - CPU'da da çalışır)
print(f"CUDA kullanılabilir mi: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = "cuda"
else:
    print("CPU kullanılacak (TinyLlama için yeterli)")
    device = "cpu"

print(f"\n✓ Model yüklenecek: {model_name}")
print(f"✓ Device: {device}")

In [ ]:
# Model yükleme
print("📦 Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("📦 Model yükleniyor...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
)

if device == "cpu":
    model = model.to(device)

print("\n✅ Model başarıyla yüklendi!")

In [ ]:
# Prompt'a cevap üretme fonksiyonu
def generate_response(prompt, max_new_tokens=256, temperature=0.7, top_p=0.9):
    """
    Verilen prompt'a model cevabı üretir.
    
    Args:
        prompt: Kullanıcı mesajı
        max_new_tokens: Maksimum üretilecek token sayısı
        temperature: Yaratıcılık (0.1-1.0, yüksek = daha yaratıcı)
        top_p: Nucleus sampling parametresi
    """
    # TinyLlama chat format
    chat_prompt = f"""<|system|>
Sen yardımcı bir asistansın. Türkçe konuşuyorsun.</s>
<|user|>
{prompt}</s>
<|assistant|>
"""
    
    # Tokenize et
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode - sadece yeni üretilen kısmı al
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    return response.strip()

print("✓ generate_response() fonksiyonu hazır!")

## Test Alanı
Aşağıdaki hücreyi çalıştırarak modelinizi test edin. İstediğiniz prompt'u yazabilirsiniz.

In [ ]:
# Elle prompt yazın ve çalıştırın
user_prompt = "Merhaba! Sen kimsin?"

print(f"Prompt: {user_prompt}")
print("\nCevap üretiliyor...\n")

response = generate_response(user_prompt)

print(f"Model Cevabı:\n{response}")

In [ ]:
# Satranç bağlamında test
chess_prompt = """Ben şu anda bir satranç oyunu oynuyorum. 
Son hamleler: 1. e4 e5 2. Nf3 Nc6
Rakibim ile dostça konuşmak istiyorum. Bana bir şeyler söyler misin?"""

print("Satranç bağlamında test:\n")
response = generate_response(chess_prompt, max_new_tokens=150)
print(f"Model Cevabı:\n{response}")

## Notlar

### TinyLlama Avantajları:
- **Küçük**: Sadece 1.1B parametre (~2.2 GB)
- **Hızlı**: CPU'da bile çalışır
- **Kolay**: Token veya özel erişim gerektirmez
- **Ücretsiz**: Hugging Face'ten direkt indirilebilir

### Sistem gereksinimleri:
- **RAM**: 4-8 GB yeterli
- **GPU**: Opsiyonel (varsa daha hızlı)
- **Disk**: ~3 GB model dosyası

### Parametreler:
- `temperature`: 0.1-1.0 (düşük = tutarlı, yüksek = yaratıcı)
- `max_new_tokens`: Maksimum cevap uzunluğu (token sayısı)
- `top_p`: 0.1-1.0 (nucleus sampling)

### Model performansı:
- Küçük model olduğu için büyük modeller kadar güçlü değil
- Basit sohbet ve talimatlar için yeterli
- Satranç sohbeti için uygun